In [15]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import random

# Load the data
file_path = 'dataset/HealthData.csv'
data = pd.read_csv(file_path)

# Preprocess the data
data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['timestamp'])
data_cleaned = data.copy()
for col in ['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']:
    data_cleaned[col] = pd.to_numeric(data_cleaned[col], errors='coerce')
data_cleaned = data_cleaned.dropna()

# Calculate Heart Rate Variability (HRV)
data_cleaned['hrv'] = data_cleaned['heartrate'].diff()

# Correlation matrix
correlation_matrix = data_cleaned[['heartrate', 'blood_oxygen_level', 'body_temperature', 'air_quality', 'room_temperature', 'hydration_levels']].corr()

# Summary Statistics
summary_stats = {
    'Average Heart Rate (BPM)': round(data_cleaned['heartrate'].mean(), 2),
    'Average Blood Oxygen Level (%)': round(data_cleaned['blood_oxygen_level'].mean(), 2),
    'Average Body Temperature (°C)': round(data_cleaned['body_temperature'].mean(), 2),
    'Average Room Temperature (°C)': round(data_cleaned['room_temperature'].mean(), 2),
    'Average Hydration Levels (%)': round(data_cleaned['hydration_levels'].mean(), 2),
    'Total Falls Detected': int(data_cleaned['fall_detection'].sum()),
    'Overall Health Score': round(((data_cleaned['heartrate'].mean() * 0.3) +
                                  (data_cleaned['blood_oxygen_level'].mean() * 0.3) +
                                  (data_cleaned['body_temperature'].mean() * 0.2) +
                                  (data_cleaned['hydration_levels'].mean() * 0.2)), 2)
}

# Healthy range comparison data
healthy_ranges = {
    'Heart Rate (BPM)': '60-100',
    'Blood Oxygen Level (%)': '95-100',
    'Body Temperature (°C)': '36.1-37.2',
    'Room Temperature (°C)': '18-21',
    'Hydration Levels (%)': '50-60'
}

# Dash App Init
app = dash.Dash(__name__)

# Live chart state variables
stream_data = pd.DataFrame({
    "datetime": pd.to_datetime([]),
    "heartrate": pd.Series(dtype='float')
})
stream_data_oxygen = pd.DataFrame({
    "datetime": pd.to_datetime([]),
    "blood_oxygen_level": pd.Series(dtype='float')
})
stream_index = 0


# Summary Layout
summary_layout = html.Div([
    html.H3("Summary Statistics", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Ul([
        html.Li(f"Average Heart Rate (BPM): {summary_stats['Average Heart Rate (BPM)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Blood Oxygen Level (%): {summary_stats['Average Blood Oxygen Level (%)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Body Temperature (°C): {summary_stats['Average Body Temperature (°C)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Room Temperature (°C): {summary_stats['Average Room Temperature (°C)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Average Hydration Levels (%): {summary_stats['Average Hydration Levels (%)']}", style={'margin-bottom': '10px'}),
        html.Li(f"Total Falls Detected: {summary_stats['Total Falls Detected']}", style={'margin-bottom': '10px'}),
        html.Li(f"Overall Health Score: {summary_stats['Overall Health Score']}", style={'margin-bottom': '10px'}),
    ], style={'font-size': '18px', 'font-family': 'Arial, sans-serif', 'color': '#34495E'})
], style={
    'border': '1px solid #e7e7e7',
    'border-radius': '10px',
    'padding': '30px',
    'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)',
    'background': 'linear-gradient(135deg, #f7f9fc, #e3e7ed)',
    'width': '80%',
    'margin': '0 auto',
    'text-align': 'left'
})

# Healthy Comparison Table
healthy_comparison_layout = html.Div([
    html.H3("Comparison with Healthy Standard Ranges (Elderly)", style={
        'text-align': 'center',
        'margin-bottom': '20px',
        'font-family': 'Georgia, serif',
        'font-size': '28px',
        'color': '#2C3E50',
        'text-transform': 'uppercase',
        'letter-spacing': '2px',
        'border-bottom': '2px solid #CCC',
        'padding-bottom': '10px'
    }),
    html.Table([
        html.Tr([html.Th("Metric"), html.Th("Your Average"), html.Th("Healthy Range")]),
        html.Tr([html.Td("Heart Rate (BPM)"), html.Td(f"{summary_stats['Average Heart Rate (BPM)']}"), html.Td(healthy_ranges['Heart Rate (BPM)'])]),
        html.Tr([html.Td("Blood Oxygen Level (%)"), html.Td(f"{summary_stats['Average Blood Oxygen Level (%)']}"), html.Td(healthy_ranges['Blood Oxygen Level (%)'])]),
        html.Tr([html.Td("Body Temperature (°C)"), html.Td(f"{summary_stats['Average Body Temperature (°C)']}"), html.Td(healthy_ranges['Body Temperature (°C)'])]),
        html.Tr([html.Td("Room Temperature (°C)"), html.Td(f"{summary_stats['Average Room Temperature (°C)']}"), html.Td(healthy_ranges['Room Temperature (°C)'])]),
        html.Tr([html.Td("Hydration Levels (%)"), html.Td(f"{summary_stats['Average Hydration Levels (%)']}"), html.Td(healthy_ranges['Hydration Levels (%)'])])
    ], style={'width': '80%', 'margin': '0 auto', 'font-size': '18px', 'border-collapse': 'collapse'})
], style={'width': '80%', 'margin': '0 auto', 'padding': '30px', 'box-shadow': '2px 2px 10px rgba(0,0,0,0.1)', 'border-radius': '10px'})

# Visualizations
gauge_fig_heart = go.Figure(go.Indicator(
    mode="gauge+number", value=summary_stats['Average Heart Rate (BPM)'],
    title={'text': "Heart Rate (BPM)"}, gauge={
        'axis': {'range': [0, 150]},
        'steps': [{'range': [60, 100], 'color': "lightgreen"},
                  {'range': [100, 120], 'color': "yellow"},
                  {'range': [120, 150], 'color': "red"}],
        'bar': {'color': "green"}}
))

gauge_fig_oxygen = go.Figure(go.Indicator(
    mode="gauge+number", value=summary_stats['Average Blood Oxygen Level (%)'],
    title={'text': "Blood Oxygen Level (%)"}, gauge={
        'axis': {'range': [0, 100]},
        'steps': [{'range': [95, 100], 'color': "lightgreen"},
                  {'range': [90, 95], 'color': "yellow"},
                  {'range': [0, 90], 'color': "red"}],
        'bar': {'color': "green"}}
))

gauge_fig_hydration = go.Figure(go.Indicator(
    mode="gauge+number", value=summary_stats['Average Hydration Levels (%)'],
    title={'text': "Hydration Levels (%)"}, gauge={
        'axis': {'range': [0, 100]},
        'steps': [{'range': [50, 60], 'color': "lightgreen"},
                  {'range': [40, 50], 'color': "yellow"},
                  {'range': [0, 40], 'color': "red"}],
        'bar': {'color': "green"}}
))

fig_timeseries = go.Figure()
fig_timeseries.add_trace(go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['heartrate'], mode='lines', name='Heart Rate (BPM)'))
fig_timeseries.add_trace(go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['blood_oxygen_level'], mode='lines', name='Blood Oxygen Level (%)'))
fig_timeseries.add_trace(go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['body_temperature'], mode='lines', name='Body Temperature (°C)'))
fig_timeseries.update_layout(title="Time-Series Analysis", title_x=0.5, xaxis_title='Time', yaxis_title='Metrics')

timeseries_explanation = html.P("This graph shows changes in heart rate, blood oxygen levels, and body temperature over time.")

fig_box = px.box(data_cleaned, x='fall_detection', y='heartrate', color='fall_detection', title="Heart Rate by Fall Detection", points="all")
boxplot_explanation = html.P("This box plot shows your heart rate when falls were detected compared to when no falls were detected.")

fig_hrv_histogram = px.histogram(data_cleaned, x='hrv', nbins=50, title="Heart Rate Variability (HRV) Distribution")
hrv_explanation = html.P("This histogram shows how much your heart rate changes.")

fig_temp_comparison = go.Figure()
fig_temp_comparison.add_trace(go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['room_temperature'], mode='lines', name='Room Temp'))
fig_temp_comparison.add_trace(go.Scatter(x=data_cleaned['datetime'], y=data_cleaned['body_temperature'], mode='lines', name='Body Temp'))

heatmap = ff.create_annotated_heatmap(
    z=correlation_matrix.values,
    x=list(correlation_matrix.columns),
    y=list(correlation_matrix.index),
    annotation_text=correlation_matrix.round(2).values,
    colorscale='Blues'
)
heatmap_explanation = html.P("This heatmap shows how metrics relate to each other.")

# Layout
app.layout = html.Div([
    html.H1("Health Monitoring Dashboard", style={'text-align': 'center'}),
    dcc.Tabs([
        dcc.Tab(label='Summary', children=[summary_layout]),
        dcc.Tab(label='Comparison with Healthy Ranges', children=[healthy_comparison_layout]),
        dcc.Tab(label='Gauge Charts', children=[
            dcc.Graph(figure=gauge_fig_heart),
            dcc.Graph(figure=gauge_fig_oxygen),
            dcc.Graph(figure=gauge_fig_hydration)
        ]),
        dcc.Tab(label='Time-Series Analysis', children=[dcc.Graph(figure=fig_timeseries), timeseries_explanation]),
        dcc.Tab(label='Box Plot: Heart Rate by Fall Detection', children=[dcc.Graph(figure=fig_box), boxplot_explanation]),
        dcc.Tab(label='HRV Distribution', children=[dcc.Graph(figure=fig_hrv_histogram), hrv_explanation]),
        dcc.Tab(label='Room vs Body Temperature', children=[dcc.Graph(figure=fig_temp_comparison)]),
        dcc.Tab(label='Correlation Heatmap', children=[dcc.Graph(figure=heatmap), heatmap_explanation]),
        dcc.Tab(label='Live Heart Rate Stream', children=[
            dcc.Graph(id='live-heart-chart'),
            dcc.Interval(id='interval-component', interval=2000, n_intervals=0)
        ]),
        dcc.Tab(label='Live Blood Oxygen Stream', children=[
            dcc.Graph(id='live-oxygen-chart'),
            dcc.Interval(id='interval-oxygen', interval=2000, n_intervals=0)
        ])

    ])
])

# Live update callback for heart rate chart
@app.callback(
    Output('live-heart-chart', 'figure'),
    Input('interval-component', 'n_intervals')
)
def update_heart_chart(n):
    global stream_data, stream_index, data_cleaned

    if stream_index < len(data_cleaned):
        new_row = {
            'datetime': data_cleaned.iloc[stream_index]['datetime'],
            'heartrate': data_cleaned.iloc[stream_index]['heartrate']
        }
        stream_data = pd.concat([stream_data, pd.DataFrame([new_row])], ignore_index=True)
        stream_index += 1

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=stream_data['datetime'],
        y=stream_data['heartrate'],
        mode='lines+markers',
        name='Heart Rate (BPM)',
        line=dict(color='red')
    ))
    fig.update_layout(title="Live Heart Rate Monitoring", xaxis_title="Time", yaxis_title="Heart Rate (BPM)")

    return fig

# Live update callback for blood oxygen chart
@app.callback(
    Output('live-oxygen-chart', 'figure'),
    Input('interval-oxygen', 'n_intervals')
)
def update_oxygen_chart(n):
    global stream_data_oxygen, stream_index, data_cleaned

    if stream_index < len(data_cleaned):
        new_row = {
            'datetime': data_cleaned.iloc[stream_index]['datetime'],
            'blood_oxygen_level': data_cleaned.iloc[stream_index]['blood_oxygen_level']
        }
        stream_data_oxygen = pd.concat([stream_data_oxygen, pd.DataFrame([new_row])], ignore_index=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=stream_data_oxygen['datetime'],
        y=stream_data_oxygen['blood_oxygen_level'],
        mode='lines+markers',
        name='Blood Oxygen Level (%)',
        line=dict(color='blue')
    ))

    fig.update_layout(
        title="Live Blood Oxygen Level Monitoring",
        xaxis_title="Time",
        yaxis_title="Blood Oxygen Level (%)",
        yaxis=dict(range=[80, 100]),
        template="plotly_white"
    )

    return fig


# Run app
if __name__ == '__main__':
    app.run(debug=True, port=8052)

## Comprehensive Overview of the Health Monitoring Dashboard Code:

### 1. **Importing Required Libraries:**
   - **dash, dcc (dash_core_components), html (dash_html_components)**: Core libraries from Dash used to build the interactive dashboard for health monitoring.
   - **plotly.express (px) and plotly.graph_objs (go)**: Libraries from Plotly for creating interactive visualizations, including simple (express) and more detailed (graph_objs) plotting tools.
   - **pandas**: A library for data manipulation and analysis, essential for loading, cleaning, and processing health data from the CSV file.
   - **numpy**: A numerical library used to calculate Heart Rate Variability (HRV) by computing differences between heart rate measurements.

### 2. **Loading and Preprocessing the Data:**
   - **Loading the CSV File**: The code loads health monitoring data from a CSV file named `HealthData.csv`.
   - **Datetime Conversion**: The date and timestamp columns are combined into a single `datetime` column to support time-series analysis.
   - **Data Cleaning**: The key health metrics such as heart rate, blood oxygen level, body temperature, room temperature, and hydration levels are converted to numeric format. Any missing values are dropped to ensure data integrity.
   - **HRV Calculation**: Heart Rate Variability (HRV) is computed by calculating the difference between consecutive heart rate values (`data['heartrate'].diff()`).

### 3. **Calculating Key Metrics:**
   - **Summary Statistics**: 
     - Average Heart Rate (BPM)
     - Average Blood Oxygen Level (%)
     - Average Body Temperature (°C)
     - Average Room Temperature (°C)
     - Average Hydration Levels (%)
     - Total Falls Detected
   
   - **Overall Health Score**: This is calculated based on an average of key metrics (heart rate, blood oxygen level, body temperature), providing an overall view of the user’s health on the summary page.

   - **Healthy Standard Ranges**: Predefined healthy ranges for elderly individuals are included to provide a comparison against the user’s actual data.

### 4. **Initializing the Dash App:**
   - `app = dash.Dash(__name__)`: Initializes the Dash app.
   - The `app.layout` defines the structure of the dashboard and how the user interacts with different components (visualizations, tabs).

### 5. **Creating Interactive Visualizations:**
   Each visualization provides an intuitive way for elderly users to monitor and understand their health metrics:

   #### A. **Gauge Charts for Key Health Metrics**:
   - **Heart Rate (BPM)**, **Blood Oxygen Level (%)**, and **Hydration Levels (%)** are represented using gauge charts. These charts display user metrics compared to healthy standard ranges, with color-coded indicators (green = healthy, yellow = caution, red = alert).

   #### B. **Summary Page**:
   - Displays key health statistics like heart rate, blood oxygen levels, body temperature, room temperature, hydration levels, and the total number of falls detected.
   - The overall health score is also displayed, giving users a quick snapshot of their health condition.

   #### C. **Time-Series Analysis**:
   - **Graph Description**: This visualization shows changes in Heart Rate (BPM), Blood Oxygen Level (%), and Body Temperature (°C) over time.
   - **Explanation**: The blue line represents heart rate, the red line shows blood oxygen, and the green line is for body temperature, allowing the user to easily track trends in their health metrics.

   #### D. **Box Plot: Heart Rate by Fall Detection**:
   - **Graph Description**: A box plot comparing heart rate when falls are detected (orange) versus when no falls are detected (blue).
   - **Explanation**: Higher heart rates during falls may indicate physical activity or stress.

   #### E. **HRV Distribution (Heart Rate Variability)**:
   - **Graph Description**: This histogram shows the distribution of Heart Rate Variability (HRV), which measures how much the user’s heart rate changes over time.
   - **Explanation**: Higher HRV typically indicates relaxation, while lower HRV could signal stress or discomfort.

   #### F. **Room and Body Temperature Comparison**:
   - **Graph Description**: This graph shows how the room temperature (blue line) compares to the body temperature (orange line) over time.
   - **Explanation**: It helps users see if fluctuations in room temperature might affect their body temperature and overall comfort.

   #### G. **Correlation Heatmap**:
   - **Graph Description**: A heatmap showing how different health metrics (e.g., heart rate, blood oxygen level, body temperature, hydration levels) correlate with each other.
   - **Explanation**: A positive correlation means two metrics tend to increase together, while a negative correlation means one metric decreases as the other increases.

   #### H. Live Heart Rate Stream:
-**Graph Description**: This visualization simulates a real-time line chart that streams heart rate (BPM) data point by point using Dash’s dcc.Interval component. Each new point is displayed every two seconds, mimicking how a live sensor feed would appear.
-**Explanation**: The heart rate values are appended progressively from the dataset to emulate continuous monitoring. This dynamic feature provides a foundation for future integration with real-time APIs like ThingSpeak or other IoT devices.

   #### I. Live Blood Oxygen Stream:
-**Graph Description**: A real-time streaming graph tracks changes in blood oxygen levels over time using a clean, dynamic line plot interface.
-**Explanation**: This visualization mirrors the heart rate stream format for design consistency. It updates every 2 seconds, helping users monitor blood oxygen trends live without visual clutter. The minimalistic approach supports clarity while maintaining clinical utility for detecting gradual oxygen saturation changes.

### 6. **Comparison with Healthy Ranges:**
   - **Page Description**: This page provides a side-by-side comparison between the user’s average metrics and the healthy standard ranges for elderly individuals. It helps users understand how their health metrics compare with optimal values.

### 7. **Creating the Layout**:
   - The layout is defined using `html.Div` to organize the structure of the dashboard. Users can navigate between different tabs using `dcc.Tabs`.
   - Each tab contains a unique visualization (gauge charts, time-series analysis, box plot, HRV distribution, room vs. body temperature comparison, and correlation heatmap).

### 8. **Running the App**:
   - `if __name__ == '__main__':` ensures that the Dash app runs only when the script is executed directly.
   - `app.run_server(debug=True)` starts the web server, allowing users to interact with the dashboard in their browser.

### Summary of Key Features:
   - **Gauge Charts**: Visualize heart rate, blood oxygen level, and hydration against healthy ranges with easy-to-read color codes.
   - **Comparison Page**: Compares the user’s metrics against healthy standard ranges for elderly individuals, providing a clear understanding of their health status.
   - **Time-Series Analysis**: Shows how key metrics change over time, allowing users to track trends.
   - **Box Plot**: Compares heart rate during falls versus non-fall situations.
   - **HRV Histogram**: Displays fluctuations in heart rate, indicating stress or relaxation levels.
   - **Room vs. Body Temperature**: Compares room temperature with body temperature to assess comfort.
   - **Correlation Heatmap**: Helps users understand how different health metrics relate to each other.
   - **Live Heart Rate Stream**: A real-time dynamic line chart that visualizes heart rate data as if it were streaming from a live sensor. Updates every 2 seconds.
   - **Live Blood Oxygen Stream**: A real-time dynamic line chart that visualizes blood oxygen level data as if it were streaming from a live sensor. Updates every 2 seconds and mirrors the heart rate stream for visual consistency.

This dashboard is designed with elderly users in mind, offering intuitive visualizations and explanations to help them monitor their health and understand key trends.
